In [1]:
# Remove verbose logging of tensorflow and absl
# (Greatly helpful on preventing freezing ipykernel while kernel sampling process)
import warnings
from silence_tensorflow import silence_tensorflow
warnings.filterwarnings('ignore')
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
silence_tensorflow()

In [2]:
import os
# initialize workspace in code
workspace_path = os.getcwd() # text the path to the workspace folder. refer to ./backend.md for further information.
from nn_meter.builder import builder_config
builder_config.init(workspace_path)

In [3]:
# generate testcases
# from nn_meter.builder.backend_meta.fusion_rule_tester import generate_testcases
# origin_testcases = generate_testcases()

from nn_meter.builder.backend_meta.fusion_rule_tester.test_fusion_rule import FusionRuleTester
from nn_meter.builder import builder_config
from nn_meter.builder.utils import merge_prev_info

import json, logging

config = builder_config.get_module('ruletest')

tester = FusionRuleTester()
origin_testcases = tester.generate()

# save information to json file
ws_path = config['MODEL_DIR']
info_save_path = os.path.join(ws_path, "results", "origin_testcases.json")
new_testcases = merge_prev_info(new_info=origin_testcases, info_save_path=info_save_path)
os.makedirs(os.path.dirname(info_save_path), exist_ok=True)
with open(info_save_path, 'w') as fp:
    json.dump(new_testcases, fp, indent=4)
logging.keyinfo(f"Save the original testcases information to {info_save_path}")

print("New testcases:", new_testcases)

New testcases: {'BF_add_add': {'add_1': {'model': '/home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/fusion_rule_test/models/BF_add_add_add_1', 'shapes': [[28, 28, 16], [28, 28, 16]]}, 'add_2': {'model': '/home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/fusion_rule_test/models/BF_add_add_add_2', 'shapes': [[28, 28, 16], [28, 28, 16]]}, 'block': {'model': '/home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/fusion_rule_test/models/BF_add_add_block', 'shapes': [[28, 28, 16], [28, 28, 16], [28, 28, 16]]}}, 'BF_add_avgpool': {'add': {'model': '/home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/fusion_rule_test/models/BF_add_avgpool_add', 'shapes': [[28, 28, 16], [28, 28, 16]]}, 'avgpool': {'model': '/home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/fusion_rule_test/models/BF_add_avgpool_avgpool', 'shapes': [[28, 28, 16]]}, 'block': {'model': '/home/

In [4]:
# connect to backend
from nn_meter.builder.backends import connect_backend
backend = connect_backend('ssh_tflite_cpu')

# run testcases and collect profiling results
from nn_meter.builder import profile_models
profiled_results = profile_models(backend, origin_testcases, mode='ruletest')

(nn-Meter) Profiling ...
Profiling BF_add_add_add_1.tflite ... 
Result metric: 0.035560299999999996 +- 0.003
Profiling BF_add_add_add_2.tflite ... 
Result metric: 0.0357761 +- 0.003
Profiling BF_add_add_block.tflite ... 
Result metric: 0.0722386 +- 0.01
Profiling BF_add_avgpool_add.tflite ... 
Result metric: 0.0357702 +- 0.003
Profiling BF_add_avgpool_avgpool.tflite ... 
Result metric: 0.0337183 +- 0.002
Profiling BF_add_avgpool_block.tflite ... 
Result metric: 0.0726316 +- 0.005
Profiling BF_add_concat_add.tflite ... 
Result metric: 0.0359822 +- 0.004
Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs
Profiling BF_add_concat_concat.tflite ... 
Result metric: 0.0584628 +- 0.008
Profiling BF_add_concat_block.tflite ... 
Result metric: 0.111631 +- 0.014
Profiling BF_add_conv_add.tflite ... 
Result metric: 0.0358262 +- 0.003
Profiling BF_add_conv_conv.tflite ... 
Result metric: 3.7589099999999998 +- 1.524
Profiling BF_add_conv_block.tflite ... 
Result metric: 2.44507000000000

In [5]:
# detect fusion rules
# from nn_meter.builder.backend_meta.fusion_rule_tester import detect_fusion_rule

# # determine fusion rules from profiling results
# detected_results = detect_fusion_rule(profiled_results)

from nn_meter.builder.backend_meta.utils import read_profiled_results

if isinstance(profiled_results, str):
    with open(profiled_results, 'r') as fp:
        profiled_results = read_profiled_results(json.load(fp))

tester = FusionRuleTester()
result = tester.analyze(profiled_results)

# save information to json file
from nn_meter.builder import builder_config
config = builder_config.get_module('ruletest')
ws_path = config['MODEL_DIR']
info_save_path = os.path.join(ws_path, "results", "detected_fusion_rule.json")
new_result = merge_prev_info(new_info=result, info_save_path=info_save_path)
os.makedirs(os.path.dirname(info_save_path), exist_ok=True)
with open(info_save_path, 'w') as fp:
    json.dump(new_result, fp, indent=4)
logging.keyinfo(f"Save the detected fusion rule information to {info_save_path}")

print("Result:", result)


Result: {'BF_add_add': {'obey': False}, 'BF_add_avgpool': {'obey': False}, 'BF_add_concat': {'obey': False}, 'BF_add_conv': {'obey': True}, 'BF_add_convtrans': {'obey': False}, 'BF_add_fc': {'obey': False}, 'BF_add_dwconv': {'obey': False}, 'BF_add_relu': {'obey': True}, 'BF_add_reshape': {'obey': False}, 'BF_avgpool_add': {'obey': True}, 'BF_avgpool_avgpool': {'obey': True}, 'BF_avgpool_concat': {'obey': True}, 'BF_avgpool_conv': {'obey': False}, 'BF_avgpool_convtrans': {'obey': False}, 'BF_avgpool_dwconv': {'obey': True}, 'BF_avgpool_relu': {'obey': True}, 'BF_avgpool_reshape': {'obey': True}, 'BF_concat_add': {'obey': False}, 'BF_concat_avgpool': {'obey': False}, 'BF_concat_concat': {'obey': False}, 'BF_concat_conv': {'obey': True}, 'BF_concat_convtrans': {'obey': False}, 'BF_concat_fc': {'obey': False}, 'BF_concat_dwconv': {'obey': False}, 'BF_concat_relu': {'obey': False}, 'BF_concat_reshape': {'obey': False}, 'BF_conv_add': {'obey': False}, 'BF_conv_avgpool': {'obey': False}, 'BF